In [22]:
import pandas as pd
import numpy as np
import scipy.stats as stats

In [43]:
def load_error_and_rank_metrics(algorithms, dataset, emb, path="results/"):

    labels = ['MSE', 'MAE', 'RMSE', 'PRECISION', 'RECALL', 'NDCG']
    max_metrics = []


    metrics_matrix = []
    
    for algorithm in algorithms:
            error_metrics = pd.read_csv(
                path + algorithm + "_" + dataset + "_" + emb + "_results_error.csv")
            
            rank_metrics = pd.read_csv(
                path + algorithm + "_" + dataset + "_" + emb + "_results_rank.csv")
            
            values = np.concatenate([error_metrics.mean().values, 
                                    rank_metrics.mean().values])

            metrics_matrix.append(values)
    

    df = pd.DataFrame(np.array(metrics_matrix),
                      columns=labels, index=algorithms)

    return df
    
    # return  error_metrics.mean(), rank_metrics.mean()

In [47]:
algorithms = ["A3NCF", "ALFM", "ANR", "CARL", "CARP", "D_ATTN",
              "DAML", "DeepCoNN", "HRDR", "MPCN", "NARRE",
              "ConvMF", "TARMF", "CARM", "CARL"]

load_error_and_rank_metrics(algorithms, "Digital_Music_data", "word2vec")

,MSE,MAE,RMSE,PRECISION,RECALL,NDCG
A3NCF,0.492633,0.472287,0.694012,0.799321,0.224311,0.008065
ALFM,0.492714,0.479093,0.694209,0.799321,0.224311,0.008065
ANR,0.378727,0.372666,0.608295,0.488885,0.286137,0.008065
CARL,0.324383,0.336193,0.561965,0.501966,0.331138,0.008065
CARP,0.301829,0.315592,0.541544,0.511101,0.366385,0.008065
D_ATTN,0.347469,0.347267,0.582711,0.480901,0.303541,0.008065
DAML,0.382040,0.403402,0.541493,0.685174,0.614226,0.142857
DeepCoNN,0.346628,0.384375,0.582662,0.462725,0.318944,0.008065
HRDR,0.320256,0.336097,0.558634,0.497728,0.332630,0.008065
MPCN,0.405369,0.421038,0.629690,0.525309,0.260806,0.008065
